In [26]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
import os

In [27]:
import tensorflow as tf

# Load the VGG19 model with pretrained ImageNet weights
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Show summary of all layers
vgg.summary()


Model: "vgg19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 20,024,384 (76.39 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
for layer in vgg.layers[:15]:
    layer.trainable = False

for layer in vgg.layers[15:]:
    layer.trainable = True


In [29]:
print("Trainable Layers:")
for layer in vgg.layers:
    print(f"{layer.name}: {'Trainable' if layer.trainable else 'Frozen'}")


Trainable Layers:
input_layer_2: Frozen
block1_conv1: Frozen
block1_conv2: Frozen
block1_pool: Frozen
block2_conv1: Frozen
block2_conv2: Frozen
block2_pool: Frozen
block3_conv1: Frozen
block3_conv2: Frozen
block3_conv3: Frozen
block3_conv4: Frozen
block3_pool: Frozen
block4_conv1: Frozen
block4_conv2: Frozen
block4_conv3: Frozen
block4_conv4: Trainable
block4_pool: Trainable
block5_conv1: Trainable
block5_conv2: Trainable
block5_conv3: Trainable
block5_conv4: Trainable
block5_pool: Trainable


In [30]:
def load_img(image_file):
    image = Image.open(image_file).resize((224, 224))  # ✅ Resize to 224x224
    image = np.array(image).astype(np.float32)[np.newaxis, ...] / 255.0
    return tf.convert_to_tensor(image)


def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = tf.cast(tensor, tf.uint8)
    return Image.fromarray(tensor.numpy()[0])


In [31]:
def vgg_layers(layer_names):
    outputs = [vgg.get_layer(name).output for name in layer_names]
    return tf.keras.Model([vgg.input], outputs)

def gram_matrix(tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', tensor, tensor)
    input_shape = tf.shape(tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super().__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = True

    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = outputs[:self.num_style_layers], outputs[self.num_style_layers:]
        style_outputs = [gram_matrix(output) for output in style_outputs]
        content_dict = {name: value for name, value in zip(self.content_layers, content_outputs)}
        style_dict = {name: value for name, value in zip(self.style_layers, style_outputs)}
        return {'style': style_dict, 'content': content_dict}


In [32]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
content_dir = "/content/drive/MyDrive/ContentImages"
style_dir = "/content/drive/MyDrive/StyleImages"


In [34]:
from google.colab import files

content_images = [os.path.join(content_dir, fname) for fname in os.listdir(content_dir)]
style_images = [os.path.join(style_dir, fname) for fname in os.listdir(style_dir)]

from google.colab import drive
drive.mount('/content/drive')

In [35]:
# Define extractor
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1']
content_layers = ['block5_conv2']
extractor = StyleContentModel(style_layers, content_layers)

# Loss function
def style_content_loss(outputs, style_targets, content_targets, style_weight=1e-2, content_weight=1e4):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) for name in style_outputs])
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) for name in content_outputs])
    style_loss *= style_weight / len(style_layers)
    content_loss *= content_weight / len(content_layers)
    return style_loss + content_loss

@tf.function
def train_step(image, extractor, style_targets, content_targets, optimizer):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs, style_targets, content_targets)
        loss += 30 * tf.image.total_variation(image)
    grad = tape.gradient(loss, image)
    optimizer.apply_gradients([(grad, image)])
    image.assign(tf.clip_by_value(image, 0.0, 1.0))

# Optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.02)

# Training over content-style pairs
for c_path, s_path in zip(content_images, style_images):
    content_image = load_img(c_path)
    style_image = load_img(s_path)

    print(f"Training on: {os.path.basename(c_path)} + {os.path.basename(s_path)}")

    style_targets = extractor(style_image)['style']
    content_targets = extractor(content_image)['content']
    image = tf.Variable(content_image)

    for i in range(20):  # Increase for better results
        train_step(image, extractor, style_targets, content_targets, optimizer)


Training on: C_image14.jpg + S_image14.jpg


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_44']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


Training on: C_image13.jpg + S_image15.jpg


UnidentifiedImageError: cannot identify image file '/content/drive/MyDrive/StyleImages/S_image13.avif'

In [ ]:
# Save final stylized image
final_image = tensor_to_image(image)
final_image.save("stylized_result.jpg")

# Save fine-tuned model
extractor.vgg.save("vgg19_partial_finetuned_for_nst.h5")


In [ ]:
from google.colab import files
files.download("stylized_result.jpg")
files.download("vgg19_partial_finetuned_for_nst.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
import os
import tensorflow as tf
import numpy as np
from PIL import Image

# --- Load image ---
def load_img(path):
    img = Image.open(path).resize((224, 224)).convert("RGB")
    img = np.array(img).astype(np.float32)[np.newaxis, ...] / 255.0
    return tf.convert_to_tensor(img)

# --- Dataset paths ---
content_dir = "/content/drive/MyDrive/ContentImages"
style_dir = "/content/drive/MyDrive/StyleImages"



In [2]:

valid_exts = ('.jpg', '.jpeg', '.png')

content_paths = sorted([
    os.path.join(content_dir, f) for f in os.listdir(content_dir)
    if f.lower().endswith(valid_exts)
])

style_paths = sorted([
    os.path.join(style_dir, f) for f in os.listdir(style_dir)
    if f.lower().endswith(valid_exts)
])

# --- Define model ---
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1']
content_layers = ['block5_conv2']

def gram_matrix(tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', tensor, tensor)
    input_shape = tf.shape(tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super().__init__()
        vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
        for layer in vgg.layers[:10]:
            layer.trainable = False
        for layer in vgg.layers[10:]:
            layer.trainable = True
        outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
        self.vgg = tf.keras.Model([vgg.input], outputs)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)

    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed)
        style_outputs, content_outputs = outputs[:self.num_style_layers], outputs[self.num_style_layers:]
        style_outputs = [gram_matrix(output) for output in style_outputs]
        content_dict = {name: value for name, value in zip(self.content_layers, content_outputs)}
        style_dict = {name: value for name, value in zip(self.style_layers, style_outputs)}
        return {'style': style_dict, 'content': content_dict}

# --- Loss and training ---
def style_content_loss(outputs, style_targets, content_targets, style_weight=1e-2, content_weight=1e4):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([
        tf.reduce_mean((style_outputs[name] - style_targets[name])**2)
        for name in style_outputs
    ])
    content_loss = tf.add_n([
        tf.reduce_mean((content_outputs[name] - content_targets[name])**2)
        for name in content_outputs
    ])
    style_loss *= style_weight / len(style_layers)
    content_loss *= content_weight / len(content_layers)
    return style_loss + content_loss

@tf.function
def train_step(image, extractor, style_targets, content_targets, optimizer):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs, style_targets, content_targets)
        loss += 30 * tf.image.total_variation(image)
    grad = tape.gradient(loss, extractor.vgg.trainable_variables)
    optimizer.apply_gradients(zip(grad, extractor.vgg.trainable_variables))


In [3]:

# --- Training loop ---
extractor = StyleContentModel(style_layers, content_layers)
optimizer = tf.optimizers.Adam(learning_rate=1e-4)

for c_path, s_path in zip(content_paths, style_paths):
    try:
        content_image = load_img(c_path)
        style_image = load_img(s_path)

        print(f"✅ Training on: {os.path.basename(c_path)} + {os.path.basename(s_path)}")

        style_targets = extractor(style_image)['style']
        content_targets = extractor(content_image)['content']
        image = tf.Variable(content_image)

        for i in range(10):  # You can increase steps for better fine-tuning
            train_step(image, extractor, style_targets, content_targets, optimizer)

    except Exception as e:
        print(f"❌ Skipping {c_path} + {s_path} due to error: {e}")



✅ Training on: C_image1.jpg + S_image1.jpg


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


✅ Training on: C_image10.jpg + S_image10.jpg
✅ Training on: C_image11.jpg + S_image11.jpg
✅ Training on: C_image12.jpg + S_image12.jpg
✅ Training on: C_image13.jpg + S_image14.jpg
✅ Training on: C_image14.jpg + S_image15.jpg
✅ Training on: C_image2.jpg + S_image2.jpg
✅ Training on: C_image3.jpg + S_image3.jpg
✅ Training on: C_image4.jpg + S_image4.jpg
✅ Training on: C_image5.jpg + S_image5.jpg
✅ Training on: C_image6.jpg + S_image6.jpg
✅ Training on: C_image7.jpg + S_image7.jpg
✅ Training on: C_image8.jpg + S_image8.jpg
✅ Training on: C_image9.jpg + S_image9.jpg


In [4]:
extractor.vgg.save("vgg19_finetuned_nst_model.h5")
print("✅ Model saved as vgg19_finetuned_nst_model.h5")

✅ Model saved as vgg19_finetuned_nst_model.h5
